## Murphy and Applegate
### Anomaly Detection Project

In [1]:
from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

from env import host, user, password
import acquire
import prepare

In [ ]:
df = acquire.get_cohort_curr_data()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

### Initial Prep To-Do:
- Concat 'date' and 'time' to datetime and set as index
- Convert 'start_date', 'end_date', 'created_at', 'updated_at' to datetime
- Drop 'deleted_at'
- Create new column that maps out whether DS or WebDev

In [ ]:
df = prepare.initial_prep(df)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#asdf